## Creating an Initial Population

Now we can create an initial population, by first defining the population size and then selecting from the list of recipes.




In [7]:
import json
import pprint
import random
import math
from collections import defaultdict
import copy
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
with open('data/mochi.json', 'r') as file:
    data = json.load(file)
recipes = data['recipes']
pprint.PrettyPrinter(indent=2, depth=3).pprint(recipes[0])

{ 'ingredients': [ { 'amount': 8.0,
                     'ingredient': 'fresh strawberries',
                     'unit': 'oz'},
                   { 'amount': 1.0,
                     'ingredient': 'mochiko flour, sweet rice or glutinous '
                                   'rice flour',
                     'unit': 'cup'},
                   { 'amount': 2.0,
                     'ingredient': 'sugar of choice',
                     'unit': 'tablespoons'},
                   { 'amount': 0.25,
                     'ingredient': 'cornstarch, potato starch or tapioca '
                                   'starch',
                     'unit': 'cup'},
                   { 'amount': 1.0,
                     'ingredient': 'sweetened white bean paste (shiro an)',
                     'unit': 'cup'}],
  'name': 'Strawberry Mochi',
  'rating': 4.71,
  'servings': 8}


So here we give each ingredient the rating of its recipe.
If an ingredient is used in multiple recipes, we'll take the mean of all the ratings for this ingredient.

In [ ]:
def calculate_average_ingredient_ratings(recipes):
    """Calculate the average rating for each unique ingredient."""

    average_ratings[ingredient] = data['total_rating'] / data['count']
    
    return average_ratings


average_ratings = calculate_average_ingredient_ratings(recipes)
for ingredient, rating in average_ratings.items():
    print(f"{ingredient}: {rating:.2f}")

KeyError: 'total_rating'

In [ ]:
all_ingredients = []
for i, recipe in enumerate(recipes):
  all_ingredients.extend(recipe['ingredients'])
  print(i+1, recipe['name'])
print("amount of ingredients:", len(all_ingredients))
print("amount of unique ingredients:", len(unique(all_ingredients)))

1 Strawberry Mochi
2 Fresh Mango
3 Applesauce Mochi
4 Sweet Potato
5 Raspberry Chocolate
6 Banana Chocolate
7 Tofu
8 Pumpkin
9 Savory Sweet Corn Mochi
10 Pistachio Butter
11 Black Sesame
12 Purple Sweet Potato Mochi
13 Green Tea / Matcha (with a Twist
14 Almond Milk
15 Crunchy Peanut Butter
16 Blueberry Mochi Ice Cream
17 Mugwort Mochi
18 Pandan Mochi
19 Black Sesame Mochi Muffins
20 Mango Mochi
21 Matcha Mochi Waffles
22 Ube Baked Mochi Donuts
23 Potato Mochi
24 Chocolate Mochi
25 Ube Mochi Muffins
26 Biscoff Baked Mochi Donuts
27 Nutella Mochi
28 Chocolate Mochi Cupcakes
29 Matcha Mochi Muffins
30 Ube Butter Mochi
31 Mochi Waffle Recipe
32 Mochi Bread
33 Chocolate Mochi Muffins
34 Mochi Cookies
35 Chocolate Mochi Donuts
36 Ube Mochi
37 Matcha Baked Mochi Donuts
38 Mochi Pancakes
39 Pandan Donuts
40 Milk Rice Cake
amount of ingredients: 305


In [ ]:
pprint.PrettyPrinter(indent=2, depth=2).pprint(all_ingredients)

[ {'amount': 8.0, 'ingredient': 'fresh strawberries', 'unit': 'oz'},
  { 'amount': 1.0,
    'ingredient': 'mochiko flour, sweet rice or glutinous rice flour',
    'unit': 'cup'},
  {'amount': 2.0, 'ingredient': 'sugar of choice', 'unit': 'tablespoons'},
  { 'amount': 0.25,
    'ingredient': 'cornstarch, potato starch or tapioca starch',
    'unit': 'cup'},
  { 'amount': 1.0,
    'ingredient': 'sweetened white bean paste (shiro an)',
    'unit': 'cup'},
  { 'amount': 0.5,
    'ingredient': 'mochiko flour, sweet rice or glutinous rice flour',
    'unit': 'cup'},
  {'amount': 0.6666666666666666, 'ingredient': 'ripened mango', 'unit': 'cup'},
  { 'amount': 0.6666666666666666,
    'ingredient': 'pastry cream, vanilla pudding or ice cream of choice',
    'unit': 'cup'},
  {'amount': 8.0, 'ingredient': 'unsweetened applesauce', 'unit': 'oz'},
  { 'amount': 1.0,
    'ingredient': 'mochiko flour, sweet rice or glutinous rice flour',
    'unit': 'cup'},
  { 'amount': 0.25,
    'ingredient': 'cor

In [ ]:
population_size = 20
population = random.choices(recipes, k=population_size)
pprint.PrettyPrinter(indent=2, depth=2).pprint(population)

[ {'ingredients': [...], 'name': 'Black Sesame', 'rating': 5.0, 'servings': 8},
  { 'ingredients': [...],
    'name': 'Mochi Pancakes',
    'rating': '5',
    'servings': 14},
  { 'ingredients': [...],
    'name': 'Applesauce Mochi',
    'rating': 4.37,
    'servings': 6},
  { 'ingredients': [...],
    'name': 'Applesauce Mochi',
    'rating': 4.37,
    'servings': 6},
  { 'ingredients': [...],
    'name': 'Crunchy Peanut Butter',
    'rating': 5.0,
    'servings': 10},
  { 'ingredients': [...],
    'name': 'Matcha Baked Mochi Donuts',
    'rating': 'needs rating',
    'servings': 6},
  { 'ingredients': [...],
    'name': 'Matcha Mochi Muffins',
    'rating': 'needs rating',
    'servings': 12},
  { 'ingredients': [...],
    'name': 'Blueberry Mochi Ice Cream',
    'rating': 5.0,
    'servings': 6},
  { 'ingredients': [...],
    'name': 'Biscoff Baked Mochi Donuts',
    'rating': 'needs rating',
    'servings': 6},
  { 'ingredients': [...],
    'name': 'Mochi Cookies',
    'rating': '5

## Evaluating Recipes (Fitness Function)

The following function defines how individuals are evaluated:
We chose to evaluate the recipe by calculating the mean of the rating of its ingredients. So, we sum up the rating for each ingredient, and then divide by the total amount of ingredients.

In [ ]:
def evaluate_recipes(recipes):
    """Evaluate the fitness of each recipe based on the average ingredient rating."""
    for r in recipes:
        valid_ratings = [ingredient['rating'] for ingredient in r['ingredients'] if isinstance(ingredient['rating'], (int, float))]
        
        if len(valid_ratings) > 0:  # Ensure there are valid ratings to avoid division by zero
            r['fitness'] = sum(valid_ratings) / len(valid_ratings)
        else:
            r['fitness'] = 0  # Default fitness for recipes with no valid ratings

Use this function to evaluate the initial population.

In [ ]:
# Evaluate the fitness of each recipe in the dataset
evaluate_recipes(recipes)

# Check the recipes with their fitness scores
for recipe in recipes:
    print(f"{recipe['name']} - Fitness: {recipe['fitness']:.2f}")

evaluate_recipes(population)
population = sorted(population, reverse = True, key = lambda r: r['fitness'])

Strawberry Mochi - Fitness: 4.75
Fresh Mango - Fitness: 4.93
Applesauce Mochi - Fitness: 4.65
Sweet Potato - Fitness: 4.84
Raspberry Chocolate - Fitness: 4.95
Banana Chocolate - Fitness: 4.16
Tofu - Fitness: 4.66
Pumpkin - Fitness: 4.77
Savory Sweet Corn Mochi - Fitness: 2.88
Pistachio Butter - Fitness: 4.96
Black Sesame - Fitness: 4.93
Purple Sweet Potato Mochi - Fitness: 4.74
Green Tea / Matcha (with a Twist - Fitness: 4.93
Almond Milk - Fitness: 4.93
Crunchy Peanut Butter - Fitness: 4.96
Blueberry Mochi Ice Cream - Fitness: 4.93
Mugwort Mochi - Fitness: 4.73
Pandan Mochi - Fitness: 0.00
Black Sesame Mochi Muffins - Fitness: 0.00
Mango Mochi - Fitness: 0.76
Matcha Mochi Waffles - Fitness: 0.00
Ube Baked Mochi Donuts - Fitness: 0.00
Potato Mochi - Fitness: 0.00
Chocolate Mochi - Fitness: 0.00
Ube Mochi Muffins - Fitness: 0.00
Biscoff Baked Mochi Donuts - Fitness: 0.00
Nutella Mochi - Fitness: 0.76
Chocolate Mochi Cupcakes - Fitness: 0.00
Matcha Mochi Muffins - Fitness: 0.00
Ube Butter

## Selecting recipes

Now we will choose a method for selecting recipes.

E.g., if we stick with the roulette wheel that is used in the example it would look like this:

In [ ]:
def select_recipe(recipes_copy):
  sum_fitness = sum([recipe['fitness'] for recipe in recipes_copy])
  f = random.randint(0, sum_fitness)
  for recipe in recipes_copy:
    if f < recipe['fitness']:
      return recipe
    f -= recipe['fitness']
  return recipes_copy[-1]

## Crossover and mutations

How do we choose what we want to use for crossover?

What mutations do we want to use? We can choose for example, increasing/decreasing the amount of an ingredient, substituting, adding or removing ingredients.
If we choose to substitute an ingredient, we want it to be substituted with the same type of ingredients (e.g, wets with wets, dries with dries).

In [ ]:
recipe_number = 1

def crossover_recipes(r1, r2):
  global recipe_number
  p1 = random.randint(1, len(r1['ingredients'])-1)
  p2 = random.randint(1, len(r2['ingredients'])-1)
  r1a = r1['ingredients'][0:p1]
  r2b = r2['ingredients'][p2:-1]
  r = dict()
  r['name'] = "recipe {}".format(recipe_number)
  recipe_number += 1
  r['ingredients'] = r1a + r2b
  return r

In [ ]:
def normalise_recipe(r):
  unique_ingredients = dict()
  for i in r['ingredients']:
    if i['ingredient'] in unique_ingredients:
      n = unique_ingredients[i['ingredient']]
      n['amount'] += i['amount']
    else:
      unique_ingredients[i['ingredient']] = i.copy()
  r['ingredients'] = list(unique_ingredients.values())

  sum_amounts = sum([i['amount'] for i in r['ingredients']])
  scale = 1000 / sum_amounts
  for i in r['ingredients']:
    i['amount'] = max(1, math.floor(i['amount'] * scale))